<a href="https://colab.research.google.com/github/cloudanum/50Algorithms/blob/main/chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip -q install -U "requests>=2.32.5,<3"
!pip -q install -U \
  "langchain==0.3.*" \
  "langchain-community==0.3.*" \
  "langchain-openai==0.2.*" \
  "duckduckgo-search>=6.2.6,<7" \
  sympy

!pip install -U ddgs



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [7]:
from importlib.metadata import version, PackageNotFoundError

def show_ver(pkg):
    try:
        print(f"{pkg}: {version(pkg)}")
    except PackageNotFoundError:
        print(f"{pkg}: NOT INSTALLED")

show_ver("requests")
show_ver("langchain")
show_ver("langchain-community")
show_ver("langchain-openai")
show_ver("duckduckgo-search")
show_ver("sympy")



requests: 2.32.5
langchain: 0.3.27
langchain-community: 0.3.29
langchain-openai: 0.2.14
duckduckgo-search: 6.4.2
sympy: 1.14.0


In [16]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")


Enter your OpenAI API key: ··········


In [17]:
# --------------------------------------------------------------
# 1) (Colab) Install dependencies — run once per runtime
# --------------------------------------------------------------
# If you haven't installed in this runtime yet, uncomment:
# !pip -q install -U langchain langchain-community langchain-openai ddgs duckduckgo-search sympy

# --------------------------------------------------------------
# 2) Set your OpenAI API key
# --------------------------------------------------------------


# --------------------------------------------------------------
# 3) Imports
# --------------------------------------------------------------
from pydantic import BaseModel, Field
from sympy import sympify

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import StructuredTool
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

from langchain.agents import AgentExecutor, create_react_agent

# --------------------------------------------------------------
# 4) Define tools
# --------------------------------------------------------------
class CalcInput(BaseModel):
    expression: str = Field(..., description="A math expression, e.g., 'sqrt(144)' or '2+2*5'")

def safe_calculator(expression: str) -> str:
    """Safely evaluate mathematical expressions with sympy."""
    try:
        return str(sympify(expression))
    except Exception as e:
        return f"Invalid mathematical expression: {e}"

calculator_tool = StructuredTool.from_function(
    name="Calculator",
    func=safe_calculator,
    description="Safely evaluate mathematical expressions using sympy.",
    args_schema=CalcInput,
)

search_tool = DuckDuckGoSearchRun(name="WebSearch")
tools = [calculator_tool, search_tool]

# --------------------------------------------------------------
# 5) LLM
# --------------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --------------------------------------------------------------
# 6) Pull the official ReAct prompt from LangChain Hub
#    (solves the agent_scratchpad/tools formatting automatically)
# --------------------------------------------------------------
prompt = hub.pull("hwchase17/react")  # requires internet in Colab

# --------------------------------------------------------------
# 7) Build the ReAct agent and executor
# --------------------------------------------------------------
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --------------------------------------------------------------
# 8) Invoke (no need to pass agent_scratchpad)
# --------------------------------------------------------------
result = agent_executor.invoke({
    "input": "What’s the square root of 144, and can you find recent news about that number?"
})

print("\n=== Final Answer ===")
print(result["output"])




> Entering new AgentExecutor chain...
To answer the question, I need to calculate the square root of 144 first. After that, I will search for recent news related to the number 144. 

Action: Calculator  
Action Input: "sqrt(144)"  12I have calculated the square root of 144, which is 12. Now, I will search for recent news related to the number 144.

Action: WebSearch  
Action Input: "news about 144"  June 14, 2025 - In 2024, around 23,200 Witnesses ... of the 144,000, an increase of nearly 14,000 since 2005. The Watch Tower Society has stated that the number of partakers is expected to decrease each year, but has also stated that members should not be concerned about the number ... June 10, 2025 - It is also a square (12·12=144). March 16, 2025 - 144 may refer to: 144 (number), the natural number following 143 and preceding 145 · AD 144 , a year of the Julian calendar, in the second century AD · 144 BC, a year of the pre-Julian Roman calendar · 144 (film), a 2015 Indian comedy · 144 (